In [1]:
# =========================
# CS-340 Project Two: Dashboard
# =========================

# Setup Jupyter Dash
from jupyter_dash import JupyterDash

# Dash components
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import base64, os

# Data & plotting
import pandas as pd
import plotly.express as px
from typing import Optional

# ------------------------
# CRUD module import (tries common file names)
# ------------------------
try:
    from animal_shelter import AnimalShelter
except ModuleNotFoundError:
    try:
        from animal_Shelter import AnimalShelter
    except ModuleNotFoundError:
        from animail_shelter import AnimalShelter  # last-resort fallback

# ------------------------
# Credentials
# ------------------------
# Set True ONLY if your grader requires aacuser/SNHU1234.
USE_GRADER_CREDS = True

if USE_GRADER_CREDS:
    username = "aacuser"
    password = "SNHU1234"
else:
    # Your working creds (the ones you've been using)
    username = "root"
    password = "v5MXK0nF6I"

# Connect to MongoDB through your CRUD module
db = AnimalShelter(username, password)

# ------------------------
# Helpers
# ------------------------
PREFERRED_COLS = [
    "animal_id","name","animal_type","breed",
    "sex_upon_outcome","age_upon_outcome_in_weeks",
    "outcome_type","date_of_birth","location_lat","location_long"
]

def fetch_df(query: Optional[dict] = None) -> pd.DataFrame:
    """Read from Mongo with your module, return a tidy DataFrame."""
    data = db.read(query or {})
    df = pd.DataFrame(data)
    if df.empty:
        return df
    if "_id" in df.columns:
        df = df.drop(columns=["_id"])
    keep = [c for c in PREFERRED_COLS if c in df.columns]
    return df[keep] if keep else df

def rx(breed_name: str) -> dict:
    """Case-insensitive regex matcher for breed (also matches '... Mix')."""
    return {"breed": {"$regex": breed_name, "$options": "i"}}

# Rescue queries per spec (age in weeks; 2 years ≈ 104 weeks)
RESCUE_QUERIES = {
    "Water Rescue": {
        "animal_type": "Dog",
        "$or": [rx(x) for x in ["Labrador Retriever","Chesapeake Bay Retriever","Newfoundland"]],
        "sex_upon_outcome": "Intact Female",
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
    },
    "Mountain or Wilderness Rescue": {
        "animal_type": "Dog",
        "$or": [rx(x) for x in ["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]],
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
    },
    "Disaster or Individual Tracking": {
        "animal_type": "Dog",
        "$or": [rx(x) for x in ["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]],
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
    },
    "Reset": {}
}

# Initial data
df = fetch_df(RESCUE_QUERIES["Reset"])
cols_for_table = list(df.columns) if not df.empty else PREFERRED_COLS

# ------------------------
# Build the app layout
# ------------------------
app = JupyterDash(__name__)

app.layout = html.Div([
    # ===== HEADER with logo + your name =====
    html.Div([
        html.Img(src="/assets/grazioso_logo.png",
                 style={"height":"60px","marginRight":"12px"}),
        html.Div([
            html.H1("Grazioso Salvare — AAC Rescue Dashboard", style={"margin":"0"}),
            html.Div("Built by: Abdelrahman Gomaa", style={"opacity":0.75})
        ])
    ], style={"display":"flex","alignItems":"center","gap":"10px","marginBottom":"12px"}),

    html.Hr(),

    # ===== Filter control =====
    html.Div([
        html.Label("Rescue filter:", style={"fontWeight":"600","marginRight":"8px"}),
        dcc.Dropdown(
            id="filter-type",
            options=[
                {"label":"Water Rescue", "value":"Water Rescue"},
                {"label":"Mountain or Wilderness Rescue", "value":"Mountain or Wilderness Rescue"},
                {"label":"Disaster or Individual Tracking", "value":"Disaster or Individual Tracking"},
                {"label":"Reset (Unfiltered)", "value":"Reset"},
            ],
            value="Reset",
            clearable=False,
            style={"width":"420px"}
        ),
    ]),

    html.Hr(),

    # ===== Data table =====
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in cols_for_table],
        data=df.to_dict('records'),
        page_size=10,
        sort_action="native",
        row_selectable="single",
        selected_rows=[0],
        style_table={"overflowX":"auto"},
        style_cell={"textAlign":"left","minWidth":"110px"}
    ),

    html.Br(),
    html.Hr(),

    # ===== Charts row =====
    html.Div(className='row', style={'display':'flex','gap':'12px','flexWrap':'wrap'}, children=[
        html.Div(id='graph-id', className='col s12 m6', style={"flex":"1","minWidth":"360px"}),
        html.Div(id='map-id',   className='col s12 m6', style={"flex":"1","minWidth":"360px"}),
    ])
])

# ------------------------
# Callbacks (Controller)
# ------------------------

# 1) Filter the DataTable with MongoDB queries
@app.callback(
    Output('datatable-id','data'),
    Input('filter-type', 'value')
)
def update_dashboard(filter_type):
    query = RESCUE_QUERIES.get(filter_type, RESCUE_QUERIES["Reset"])
    dff = fetch_df(query)
    return dff.to_dict('records')

# 2) Chart (Top Breeds bar) based on the rows currently visible in the table
@app.callback(
    Output('graph-id', "children"),
    Input('datatable-id', "derived_virtual_data")
)
def update_graphs(viewData):
    cur = pd.DataFrame.from_dict(viewData) if viewData is not None else df.copy()

    if cur.empty or "breed" not in cur.columns:
        fig = px.bar(title="Top Breeds (no data)", template="plotly_white",
                     pattern_shape_sequence=[""])
    else:
        breeds = cur["breed"].astype(str).fillna("Unknown")
        top = breeds.value_counts(dropna=False).head(10).reset_index()
        top.columns = ["breed", "count"]

        fig = px.bar(
            top,
            x="breed",
            y="count",
            title="Top Breeds",
            template="plotly_white",
            pattern_shape_sequence=[""]  # keep simple to avoid pattern bug
        )
        fig.update_layout(
            xaxis_title="",
            yaxis_title="Count",
            margin=dict(l=20, r=20, t=40, b=20),
            bargap=0.2
        )

    return [dcc.Graph(figure=fig, config={"displayModeBar": False})]

# 3) Highlight selected columns
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    Input('datatable-id', 'selected_columns')
)
def update_styles(selected_columns):
    selected_columns = selected_columns or []
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# 4) Update geolocation map based on selected row
@app.callback(
    Output('map-id', "children"),
    Input('datatable-id', "derived_virtual_data"),
    Input('datatable-id', "derived_virtual_selected_rows")
)
def update_map(viewData, index):
    if viewData is None:
        return []

    dff = pd.DataFrame.from_dict(viewData)
    if dff.empty:
        return [dl.Map(style={'width':'100%','height':'500px'}, center=[30.75,-97.48], zoom=10,
                       children=[dl.TileLayer(id="base-layer-id")])]

    row = (index or [0])[0]
    row = max(0, min(row, len(dff)-1))

    def _get(col):
        return dff.iloc[row][col] if col in dff.columns else None

    lat = float(_get("location_lat")) if pd.notna(_get("location_lat")) else None
    lon = float(_get("location_long")) if pd.notna(_get("location_long")) else None
    breed = str(_get("breed")) if _get("breed") is not None else "Unknown"
    name  = str(_get("name"))  if _get("name")  is not None else "Unknown"

    center = [lat, lon] if (lat is not None and lon is not None) else [30.75, -97.48]  # Austin

    marker_children = []
    if lat is not None and lon is not None:
        marker_children = [
            dl.Tooltip(breed),
            dl.Popup([html.H4("Animal Name"), html.P(name)])
        ]

    return [
        dl.Map(style={'width':'100%','height':'500px'}, center=center, zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=marker_children)
                if (lat is not None and lon is not None) else dl.LayerGroup()
        ])
    ]

# ------------------------
# Run the app
# ------------------------
# You used this port successfully before—keep it consistent.
app.run_server(mode="inline", debug=False, port=30067)


 * Running on http://127.0.0.1:30067/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Aug/2025 02:29:21] "GET /_alive_a02e57b7-c6f9-4db9-9f9e-4208637ee744 HTTP/1.1" 200 -


MongoDB connection established.


In [2]:
# Kill any leftover JupyterDash servers
try:
    from jupyter_dash import JupyterDash
    JupyterDash._terminate_servers()
    print("Old Dash servers terminated.")
except Exception as e:
    print("Cleanup skipped:", e)

Cleanup skipped: type object 'JupyterDash' has no attribute '_terminate_servers'


In [3]:
import os
print(os.path.exists("assets/grazioso_logo.png"))


127.0.0.1 - - [16/Aug/2025 02:29:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2025 02:29:21] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_8_1m1675174511.12.1.min.js HTTP/1.1" 200 -


True


127.0.0.1 - - [16/Aug/2025 02:29:21] "GET /_dash-component-suites/dash/deps/react@16.v2_8_1m1675174511.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2025 02:29:21] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_8_1m1675174511.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2025 02:29:21] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_8_1m1675174511.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2025 02:29:21] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_8_0m1675174511.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2025 02:29:21] "GET /_dash-component-suites/dash_leaflet/dash_leaflet.v0_1_23m1636397671.min.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2025 02:29:21] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_8_1m1675174511.min.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2025 02:29:21] "GET /_dash-component-suites/dash/dcc/dash_core_components-shared.v2_8_0m1675174511.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2025 02:29:21] "GET /_da